# Twitter Sentiment Analysis

1. Twitter Sentiment Analysis is the process of analyzing text data and sorting it into sentiments positive, negative, or neutral. Using sentiment analysis  to analyze opinions in Twitter data can help companies understand how people are talking about their brand.

In [133]:
import numpy as np # linear algebra
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [134]:
data = pd.read_csv('Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]
print(len(data))

13871


In [166]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

#print(data[ data['sentiment'] == 'Positive'].size)
#print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
batch_size = 15500
tokenizer = Tokenizer(nb_words=batch_size, split=' ')

tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

maxx = 0
c=0
for ll in X:
    c=len(ll)
    if c>maxx:
        maxx=c
print('Max',maxx)
X = pad_sequences(X)
print(len(X[0]))

C:\Users\webfurther\Anaconda3\lib\site-packages\keras_preprocessing\text.py:174: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Max 29
29


In [167]:
embed_dim = 128
lstm_out = 128
model = Sequential()
model.add(Embedding(batch_size, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Users\webfurther\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
C:\Users\webfurther\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 29, 128)           1984000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 2,115,842
Trainable params: 2,115,842
Non-trainable params: 0
_________________________________________________________________
None


In [168]:
print(data['sentiment'].values)
Y = pd.get_dummies(data['sentiment']).values
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

['Positive' 'Positive' 'Positive' ... 'Positive' 'Negative' 'Positive']
[[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [1 0]
 [0 1]]
(7188, 29) (7188, 2)
(3541, 29) (3541, 2)


In [169]:
model.fit(X_train, Y_train, nb_epoch = 2, batch_size=32, verbose = 2)

C:\Users\webfurther\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/2
 - 33s - loss: 0.4308 - acc: 0.8180
Epoch 2/2
 - 30s - loss: 0.2517 - acc: 0.8994


In [170]:
z=np.array(X_test[9]).reshape(1,29)

In [171]:
model.predict(z)

array([[0.99799156, 0.00200847]], dtype=float32)

In [172]:
Y_test[9]

array([1, 0], dtype=uint8)

In [179]:
s1 = "john is a nice guy you would love to work with him"
s2 = "that was a bummer and an epic fail movie just dont go to watch"
hh = tokenizer.texts_to_sequences([s1])


In [180]:
hh

[[430, 5, 7, 701, 358, 11, 72, 218, 4, 471, 45, 74]]

In [181]:
X_test[9].shape

(29,)

In [182]:
hh = pad_sequences(hh, maxlen=29)

In [183]:
hh=np.array(hh[0]).reshape(1,29)

In [184]:
model.predict(hh)

array([[0.18184733, 0.8181526 ]], dtype=float32)